## Trying to find methane plumes using Landsat 8 or Sentinel-2 from a list of ones already found (csv file from Fei)

07/02/24
Using "data_esthag.csv" from https://pubs.acs.org/doi/full/10.1021/acs.est.1c08575

### Import libraries

In [7]:
import os

import ee
ee.Authenticate()

#Initialize earth engine for a new Python session

ee.Initialize()

import geemap

from geemap import *

In [8]:
%pip install xarray


Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import xarray as xr # May need to install using %pip install xarray
import datetime

### Looking for plume:
date: 12/12/2017  06:42:49


#### With Landsat 8

In [10]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2017-12-12', '2017-12-13')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B11', 'B12'],
}

Map.setCenter(36.6, 61.6, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[61.6, 36.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [11]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2017-12-12', '2017-12-13')
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['SR_B6', 'SR_B7'],
}

Map.setCenter(36.6, 61.6, 12)
Map.addLayer(image, vis, 'Landsat 8')
Map

Map(center=[61.6, 36.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

#### With Sentinel-2

### Search for an orbit that passes over a plume
Do this by visualising Landsat and Sentinel data over the plume location and time, and seeing if the swath passes over. Write simple code to extract the dates and locations from data_esthag.csv to search by plume (basically with the row number as the input)

In [12]:
import csv
import pandas as pd

In [13]:
# Read inn plumes csv file and convert into dataframe
plumes = pd.read_csv('data_esthag.csv')
plumesDF = pd.DataFrame(plumes)
#plumesDF

In [14]:
# Make an array (just_date) of the dates from dataframe date column, without the times

just_date = [] # Empty array to store dates in
date_detection = plumes.loc[:, 'date']
for d in date_detection:
    just_date.append(d[0:10])



#### Project Satellite data for day of chosen plume (select plume by row number)


In [15]:
from datetime import datetime as dt
from datetime import timedelta

In [16]:
# Select date and location according to plume (row number)

row_n = 282
print('Plume number:', row_n)
date_str = just_date[row_n]
# Get date of plume
date_str = just_date[row_n]
date_start = dt.strptime(date_str, '%d/%m/%Y')
print('Plume date:',date_start)
# Get date of next day
date_end = date_start + timedelta(1)


# Get source latitude and longitude to 2dps
string = plumes.loc[:, 'source'][row_n]

source_lat = round(float(string[1:-1].split(',')[0]), 2)
source_lon = round(float(string[1:-1].split(',')[1]), 2)
print('Plume source at (', source_lat, ',',source_lon, ')')

# Reformat the dates to go in the image collection search

plume_date_start = date_start.strftime('%Y-%m-%d')
plume_date_end = date_end.strftime('%Y-%m-%d')

#### Sentinel-2

Plume number: 282
Plume date: 2020-09-12 00:00:00
Plume source at ( 28.7 , 7.6 )


In [17]:
type(plume_date_start)
plume_date_start

'2020-09-12'

#### Sentinel-2

In [18]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(plume_date_start, plume_date_end)
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 7000,
    #'bands': ['B11', 'B12'],
    'bands': ['B12']
}

Map.setCenter(source_lat, source_lon, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[7.6, 28.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [19]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(plume_date_start, plume_date_end)
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 7000,
    #'bands': ['B11', 'B12'],
    'bands': ['B11']
}

Map.setCenter(source_lat, source_lon, 15)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[7.6, 28.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

#### Landsat

In [20]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate(plume_date_start, plume_date_end)
)
image = collection.median()

vis = {
    #'min': 0.0,
    #'max': 3000,
    'bands': ['SR_B6', 'SR_B7'],
    #'bands': ['SR_B6']
    #'bands': ['SR_B7']
}

Map.setCenter(source_lat, source_lon, 3)
Map.addLayer(image, vis, 'Landsat 8')
Map

Map(center=[7.6, 28.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

#### NEXT STEPS (8/2/24)
- Look at paper to see what satellites are used to detect these plumes. Can't be S2 and Landsat for all of them
- Code a lign to mark the lat/lon centre point on the map
- zoom - how does it work and what is the scaling factor it uses?

## Finding Hassi Messaoud plume from Varon (2021) paper
file:///C:/Users/s1709837/OneDrive%20-%20University%20of%20Edinburgh/Documents/Papers%20and%20Reports/varon_2021.pdf

Source: 31.6585◦ N, 5.9053◦ E
Date: 20/11/2019

ImageCollection package help:
https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate

### Set up plot, visualise B12 over relevant area and date

In [21]:
from datetime import datetime as dt
from datetime import timedelta

#### Define date and location of plume source

In [111]:
# Define date and location of Hassi Messaoud plume

source_lat = 31.6585
source_lon = 5.9053
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2019-11-20'
plume_date_end = '2019-11-21'
print('Plume detection date:' , plume_date_start)


Plume source at ( 31.6585 , 5.9053 )
Plume detection date: 2019-11-20


#### Make a rectangle over area over which Varon 2021 images the Hassi Messaoud plume

In [112]:
# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = 5.88419
lonmax = 5.92609
latmin = 31.64011
latmax = 31.67640

# Set up GEE geometry shape
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

#### Plot Sentinel-2 data over this area for day of plume

#### Load image for S2 L-1C (TOA) Hassi Messaoud on 20/11/2019

In [113]:
# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

In [114]:
# Find out whether it's Sentinel 2A or 2B
# List of image properties: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#image-properties
img.get('SPACECRAFT_NAME')

# Therefore, model m should be  mMBSP = −0.029 for Sentinel-2A

#### Plot Band 12 TOA reflectance over Hassi Massaoud

In [115]:
# Plot only B12 to see plume as show in Varon (2021) Figure 2b
mapB12 = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 0.35,
    'max': 0.75,   
    'bands': ['B12']
   
}

mapB12.setCenter(source_lon, source_lat, 13.5) # Define map centre
mapB12.addLayer(img.divide(10000), vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
# Map.remove_colorbar(clear)
mapB12.add_colorbar(vis, label='TOA Refelctance')

mapB12

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [116]:
# What is the max value of B12?



#### Least squares fitting (linear fit) https://developers.google.com/earth-engine/guides/reducers_regression#linearfit_2

In [117]:
# Linear fit for a region, applying reduceRegion to ee.Image as laid out by https://developers.google.com/earth-engine/guides/reducers_regression

# Define geometry
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Subset the SWIR1 and SWIR2 bands. In the regression reducer, independent
 # variables come first followed by the dependent variables. In this case,
 # B5 (SWIR1) is the independent variable and B6 (SWIR2) is the dependent
 # variable.
imRegress = img.select(['B11', 'B12'])

# Calculate the regressions coefficiets for the set of pixels intersecting the above defined region using reduceRegion with ee.Reducer.linearFit()


linearFit = imRegress.reduceRegion(
    reducer = ee.Reducer.linearFit(),
    geometry = geometry_area,
   
)                          

linearFit



### Image operation

In [130]:
# Plot the enhancement found without empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP
R_MBSP = img.expression(
    '(R12 - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.029)

# Define visualisation parameters for plot
vis = { 'min': -0.3, 'max': 0.1, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### 04/03/24 - what is my plot showing? Can I extract the enhancement?

In [62]:
# Using method from https://developers.google.com/earth-engine/guides/reducers_reduce_region
# except that there is only one band here, so we are getting mean rather than mean dictionary

mean = F_MBSP.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

# print(mean.get('B11').getInfo())
mean.getInfo()

{'B12': -0.10284697666315486}

In [44]:
# Test with B12 only to see if getting same result

B12 = img.select('B12') # Get B12 only out of image
B12

meanb12 = B12.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

meanb12.getInfo()


{'B12': 5201.10148815842}

In [137]:
# Test with B12 only to see if getting same result

B12 = img.select('B12') # Get B12 only out of image
B12

maxb12 = B12.reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

maxb12.getInfo()


{'B12': 6305}

In [ ]:
img.getInfo()

In [1]:
F_MBSP.get()

NameError: name 'F_MBSP' is not defined

#### Resume work

In [138]:
# Plot same as above but with empirical scaling factor c = 0.9475287960061192 (from least squares fitting calculation above)

# Basemap
Map = geemap.Map() # plot basemap

R_MBSP = img.expression(
    '(((0.9475287960061192 * R12) - R11)/R11 )', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
        # 'R11': img.select('B11'),
        # 'R12': img.select('B12')
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.029)

# Define visualisation parameters for plot


vis = {'min': -0.35, 'max': 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']	}


# Add data layers to basemap and plot
Map.setCenter(source_lon, source_lat, 13.75) # Define map centre and zoom
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Plume masking to find IME

In [24]:
m = geemap.Map()
multi_band_mask_img = img.mask()
display('Multi-band mask image', multi_band_mask_img)
m.add_layer(multi_band_mask_img, None, 'Multi-band mask image')

single_band_mask_img = img.select('B12').mask()
display('Single-band mask image', single_band_mask_img)
m.add_layer(single_band_mask_img, None, 'Single-band mask image')
m

'Multi-band mask image'

'Single-band mask image'

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Kazakhstan blowout plume: applying above methodolody (From Varon et al. (2021)) to search for Kazakhstan plume
https://eartharxiv.org/repository/view/6709/ (Guanter 2023 preprint)
Using sentinel-2 retrieval data in Table S1 (Guanter 2023 preprint)

Source (location of blowout specified in Guanter (2023 preprint) Fig 1a:  (45.3324◦N, 52.3730◦E)
Date: 09/06/2023

ImageCollection package help:
https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate

#### Set up plot, visualise B12 over relevant area and date

In [13]:
from datetime import datetime as dt
from datetime import timedelta

#### Define date and location of plume source

In [68]:
# Define date and location of Hassi Messaoud plume

source_lat = 45.3324
source_lon = 52.3730
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2023-06-09'
plume_date_end = '2023-06-10'
print('Plume detection date:' , plume_date_start)


Plume source at ( 45.3324 , 52.373 )
Plume detection date: 2023-06-09


#### Make a rectangle over area over which Varon 2021 images the Hassi Messaoud plume

In [69]:
# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
# BLcorner = 45.276069, 52.292236
#URcorner = 45.374880, 52.435128
lonmin = 52.292236
lonmax = 52.435128
latmin = 45.276069
latmax = 45.374880

# Set up GEE geometry shape
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

#### Plot Sentinel-2 data over this area for day of plume

#### Load image for S2 L-1C (TOA) Hassi Messaoud on 20/11/2019

In [70]:
# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

In [71]:
# Find out whether it's Sentinel 2A or 2B
# List of image properties: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#image-properties
img.get('SPACECRAFT_NAME')

# Therefore, model m should be  mMBSP = −0.022 for Sentinel-2B - MAYBE THIS HAS TO CHANGE FOR THE DIFFERENT OBSERVATION?

#### Plot B12 only

In [87]:
# Plot only B12 to see plume as show in Varon (2021) Figure 2b
Map = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 2000,
    'max': 4000,   
    'bands': ['B12'],
    # "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"]
   
}

Map.setCenter(source_lon, source_lat, 14) # Define map centre
Map.addLayer(img, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Least squares fitting (linear fit) https://developers.google.com/earth-engine/guides/reducers_regression#linearfit_2

In [76]:
# Linear fit for a region, applying reduceRegion to ee.Image as laid out by https://developers.google.com/earth-engine/guides/reducers_regression

# Define geometry
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Subset the SWIR1 and SWIR2 bands. In the regression reducer, independent
 # variables come first followed by the dependent variables. In this case,
 # B5 (SWIR1) is the independent variable and B6 (SWIR2) is the dependent
 # variable.
imRegress = img.select(['B11', 'B12'])

# Calculate the regressions coefficiets for the set of pixels intersecting the above defined region using reduceRegion with ee.Reducer.linearFit()


linearFit = imRegress.reduceRegion(
    reducer = ee.Reducer.linearFit(),
    geometry = geometry_area,   
)                          

linearFit


### Image operation

In [83]:
# Plot the enhancement found without empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP
R_MBSP = img.expression(
    '(( 0.9714300514220611*R12) - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.022)

# Define visualisation parameters for plot
vis = { 'min': -0.3, 'max': 0.2}


Map.setCenter(source_lon, source_lat, 13.) # Define map centre
Map.addLayer(F_MBSP, vis,  'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [22]:
# Plot same as above but with empirical scaling factor c = 0.9475287960061192 (from least squares fitting calculation above)

# Basemap
Map = geemap.Map() # plot basemap

R_MBSP = img.expression(
    '(((0.9475287960061192 * R12) - R11)/R11 )', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11'),
        'R12': img.select('B12'),
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.022)

# Define visualisation parameters for plot
vis = { 'min': -0.25, 'max': 0.25}
# vis = { 'min': -1, 'max': 0, 'palette': ['#12436D', '#ff0000']
      # }

# Add data layers to basemap and plot
Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…